In [80]:
import unittest

class TestAnalysis(unittest.TestCase):
    def test_cumulative_sum(self):
        test_file_name = '/Users/kaleighjaeger/Scripps/epidemiology/COVID-19/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_confirmed_global.csv'
        df = load_to_memory(test_file_name)
        use_melt(df)
        print(use_melt(df))
        
        self.assertTrue(df[df['Country/Region'] == 'Afghanistan'] & df[df['Date'] == ['2020-07-06']]) == 20742904
#         self.assertEqual(df[df['Country/Region'] == 'Zimbabwe'], 0)
#             & df['Date'] == ['2020-08-13'] == 500000
        
        
unittest.main(argv=[''], verbosity=2, exit=False)

test_cumulative_sum (__main__.TestAnalysis) ... 

Using Melt Function
Using Melt Function
            Country/Region       Date   Cases  Cumulative Sum
148149         Afghanistan 2021-07-06  127464        20742904
148150             Albania 2021-07-06  132544        24802238
148151             Algeria 2021-07-06  142447        32820101
148152             Andorra 2021-07-06   13991         2905567
148153              Angola 2021-07-06   39375         6178651
...                    ...        ...     ...             ...
148423             Vietnam 2021-07-06   22341         1103416
148424  West Bank and Gaza 2021-07-06  314780        53675169
148425               Yemen 2021-07-06    6931         1222752
148426              Zambia 2021-07-06  167132        18280206
148427            Zimbabwe 2021-07-06   57963         8142767

[279 rows x 4 columns]


ERROR

ERROR: test_cumulative_sum (__main__.TestAnalysis)
----------------------------------------------------------------------
Traceback (most recent call last):
  File "/Library/Python/3.8/site-packages/pandas/core/indexes/base.py", line 3080, in get_loc
    return self._engine.get_loc(casted_key)
  File "pandas/_libs/index.pyx", line 70, in pandas._libs.index.IndexEngine.get_loc
  File "pandas/_libs/index.pyx", line 101, in pandas._libs.index.IndexEngine.get_loc
  File "pandas/_libs/hashtable_class_helper.pxi", line 4554, in pandas._libs.hashtable.PyObjectHashTable.get_item
  File "pandas/_libs/hashtable_class_helper.pxi", line 4562, in pandas._libs.hashtable.PyObjectHashTable.get_item
KeyError: 'Date'

The above exception was the direct cause of the following exception:

Traceback (most recent call last):
  File "<ipython-input-80-f426da9e68f5>", line 11, in test_cumulative_sum
    self.assertTrue(df[df['Country/Region'] == 'Afghanistan'] & df[df['Date'] == ['2020-07-06']]) == 207

In [23]:
import pandas as pd 
from pull_covid_data import pull_covid_data
from datetime import date

pull_covid_data()

def load_to_memory(filename):
    data = pd.read_csv(f'{filename}', header=0) 
    return data

cases_file = '/Users/kaleighjaeger/Scripps/epidemiology/COVID-19/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_confirmed_global.csv'
deaths_file = '/Users/kaleighjaeger/Scripps/epidemiology/COVID-19/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_deaths_global.csv'
recovered_file = '/Users/kaleighjaeger/Scripps/epidemiology/COVID-19/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_recovered_global.csv'
# dataframe = load_to_memory(time_series)

def parse_jhu_timeseries_to_dataframe(df):
    column_constant = 4
    melt_df = pd.melt(df, id_vars=['Country/Region'], value_vars=list(df.columns[column_constant:]))
    melt_df.columns = ['Country/Region', 'Date', 'Cases']
    
    #Grouping by Country/Region and adding column "Cumulative Sum" as a result of groupby
    axis_constant = 0
    melt_df['Cumulative Sum'] = melt_df.groupby(['Country/Region'])['Cases'].cumsum(axis=axis_constant)
    
    #Changing date to proper format
    melt_df['Date'] = pd.to_datetime(melt_df['Date'])
    
    # 279 is the number of unique Country/Regions for the DataFrame
    return melt_df

#Test data files 
# cases_file = '/Users/kaleighjaeger/Scripps/Test_Cases.csv'
# deaths_file = '/Users/kaleighjaeger/Scripps/Test_Deaths.csv'

def create_test_file(filename):
    data = pd.read_csv(f'{filename}', header=0)
    edited_data = data.head(1)
    return data

def merge_cases_and_deaths(cases_file, deaths_file):
    cases = parse_jhu_timeseries_to_dataframe(create_test_file(f'{cases_file}'))
    cases.columns = ['Country/Region', 'Date', 'Cases', 'Cumulative Cases']
    deaths = parse_jhu_timeseries_to_dataframe(create_test_file(f'{deaths_file}'))
    deaths.columns = ['Country/Region', 'Date', 'Deaths', 'Cumulative Deaths']
    merged_file = pd.merge(cases,deaths, on=['Date', 'Country/Region'])
    return merged_file

def append_recovered_data(recovered_file):
    recovered = parse_jhu_timeseries_to_dataframe(create_test_file(f'{recovered_file}'))
    recovered.columns = ['Country/Region', 'Date', 'Recovered', 'Cumulative Recovered']
    merged_file = merge_cases_and_deaths(cases_file, deaths_file)
    new_file = pd.merge(merged_file,recovered, on=['Date', 'Country/Region'])
    print(new_file)
    return new_file

def calculate_first_value(df):
    nonzero_constant = 0
    idx = df.Cases.ne(nonzero_constant).idxmax()
    deaths_idx = df.Deaths.ne(nonzero_constant).idxmax()
    df['dead_firstDate'] = df.loc[deaths_idx]['Date']
    df['confirmed_firstDate'] = df.loc[idx]['Date']
    return df

def calculate_first_date():
    df = append_recovered_data(recovered_file)
    df = df.groupby(['Country/Region']).apply(
        calculate_first_value
    )
    return df

def calculate_percent_increase():
    data = calculate_first_date()
    rounded_constant = 2
    data['Percent Increase (Cases)'] = data['Cases'].pct_change().round(rounded_constant)
    return data


#Incomplete function
def calculate_rolling_average():
    df = calculate_percent_increase()
    data = df.sort_values(by = ['Country/Region', 'Date'])
    rolling_average_constant = 3
    #Need to filter for each country using GroupBy 
    data['Rolling Average'] = df.groupby(['Country/Region']).rolling(rolling_average_constant, center=True)['Cases'].mean().reset_index(drop=True)
#     data['Rolling Average'] = df.groupby('Country/Region').rolling(rolling_average_constant, center=True)['Cases'].mean().reset_index(drop=True)
    return data
    


'''
Unit Testing with Pytest
'''

import pytest

def generate_new_file():
    data = calculate_percent_increase()
    data_sorted = data.sort_values(by = ['Country/Region', 'Date'])
    data_cut = data_sorted[0:500]
    data_cut.to_csv('cases_data.csv')
    
# generate_new_file()
    
def read_file_for_testing():
    cases_data = '/Users/kaleighjaeger/Scripps/epidemiology/cases_data.csv'
    return pd.read_csv(cases_data)
    


def cumulative_cases(df, value):
    return df.iloc[value]['Cumulative Cases']

def first_date(df, value):
    return df.iloc[value]['confirmed_firstDate']

# def percent_increase(df, value):
#     return df.iloc[value]['Percent Increase (Cases)']
  
    
def test_cumulative_cases():
    df = read_file_for_testing()
    assert cumulative_cases(df, 0) == 0
    assert cumulative_cases(df, 499) == 17467791
    
def test_first_date():
    df = read_file_for_testing()
    assert first_date(df, 0) == '2020-02-24'

# def test_percent_increase():
#     df = read_file_for_testing()
#     assert percent_increase(df, 0) == 0
#     assert percent_increase(df, 10) == 0.20
    
def run_all_tests():
    test_cumulative_cases()
    test_first_date()
    test_percent_increase()

run_all_tests()


Here is the status update of COVID-19: 

On branch master
Your branch is up to date with 'origin/master'.

nothing to commit, working tree clean


AssertionError: 